# Tarea Pyvista 
Manuel Jiménez Haro

Tema 3 - Procesado de Mallas con PyVista (interior vs exterior de una malla)

Proyecto de PyVista para procesamiento de mallas:

###  1) Cargar Malla, Decimar, Suavizar

In [1]:
import pyvista as pv
pv.set_jupyter_backend('ipyvtklink')  
import numpy as np

In [2]:
#cargar malla
mesh = pv.read('PK6.vtk')

mesh.plot(show_edges=True)

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

In [3]:
#decimar
target_reduction = 0.7
print(f"Reducing {target_reduction * 100.0} percent out of the original mesh")

decimated = mesh.decimate(target_reduction)
decimated.plot(show_edges=True)

Reducing 70.0 percent out of the original mesh


ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

In [4]:
#suavizar
# Get the out surface as PolyData
surf = decimated.extract_geometry()
# Smooth the surface
smooth = surf.smooth(n_iter=1000)
smooth.plot(show_edges=True)

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

### 2) Calcular y Visualizar el campo de vectores normales a las caras

In [5]:
# calculamos los vectores normales
norm=smooth.compute_normals()
norm


PolyData (0x1e529421400)
  N Cells:	12643
  N Points:	6377
  X Bounds:	-2.414e+01, 4.745e+01
  Y Bounds:	-2.248e+01, 5.939e+01
  Z Bounds:	-3.721e+01, 2.538e+01
  N Arrays:	2

In [6]:
norm.plot_normals(mag=3,faces=True, flip=True)

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

### 3) Análisis del interior de la malla:

#### 3.1) Rellenado del interior con puntos y visualización

In [20]:
smooth

PolyData,Information
N Cells,12643
N Points,6377
X Bounds,"-2.414e+01, 4.745e+01"
Y Bounds,"-2.248e+01, 5.939e+01"
Z Bounds,"-3.721e+01, 2.538e+01"
N Arrays,0


In [21]:
# Create random XYZ points
points = np.random.rand(10000, 3)
# determinamos el espacio
points[:,0] = points[:,0]*(2.414e+01 + 4.745e+01)-2.414e+01
points[:,1] = points[:,1]*(2.248e+01 + 5.939e+01)-2.248e+01
points[:,2] = points[:,2]*(3.721e+01 + 2.538e+01)-3.721e+01

In [22]:
# Make PolyData
point_cloud = pv.PolyData(points)
point_cloud

PolyData,Information
N Cells,10000
N Points,10000
X Bounds,"-2.414e+01, 4.744e+01"
Y Bounds,"-2.248e+01, 5.937e+01"
Z Bounds,"-3.721e+01, 2.538e+01"
N Arrays,0


In [23]:
plotter = pv.Plotter()
plotter.add_mesh(point_cloud, color='g')
plotter.add_mesh(smooth, color='r')

# plotter.camera_position = 'yz'
plotter.show()

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

#### 3.2) Filtrado de puntos que caen fuera de la malla

In [24]:
selected = point_cloud.select_enclosed_points(smooth, check_surface=False)
pts = point_cloud.extract_points(selected['SelectedPoints'].view(bool),
                           adjacent_cells=False)
pl = pv.Plotter()
_ = pl.add_mesh(smooth, style='wireframe')
_ = pl.add_points(pts, color='g')
pl.show()

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)